In [ ]:
#is_copied=0
is_copied=1
bandwidth=0.01
#bandwidth=0
batch_size=256
#batch_size=128
center=5
#center=1
latent_dim=64
pou='indicator'
gamma=10
#pou='smooth'
penalty="MMD"
#penalty="SWASS"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
  
  
from matplotlib import pyplot as plt
import math, os
from numpy import array as matrix, arange
from numpy import zeros, ones,empty
import tensorflow_datasets
import random
import math
import sklearn
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from tensorflow.keras import datasets, layers, models,constraints
import os
import zipfile

import gdown

from torch.utils.data import Dataset
from natsort import natsorted
 
from PIL import Image

In [ ]:
from tensorflow.keras import datasets, layers, models,constraints
import os
import zipfile

import gdown

from torch.utils.data import Dataset
from natsort import natsorted
 
from PIL import Image

In [ ]:
!mkdir celeba && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip 
import zipfile

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("celeba/")

In [ ]:
import os
root = './celeba/img_align_celeba'
img_list = os.listdir(root)
print(len(img_list))

In [ ]:
class CelebADataset(Dataset):
  def __init__(self, root_dir, transform=None):
    """
    Args:
      root_dir (string): Directory with all the images
      transform (callable, optional): transform to be applied to each image sample
    """
    # Read names of images in the root directory
    image_names = os.listdir(root_dir)

    self.root_dir = root_dir
    self.transform = transform 
    self.image_names = natsorted(image_names)

  def __len__(self): 
    return len(self.image_names)

  def __getitem__(self, idx):
    # Get the path to the image 
    img_path = os.path.join(self.root_dir, self.image_names[idx])
    # Load image and convert it to RGB
    img = Image.open(img_path).convert('RGB')
    # Apply transformations to the image
    if self.transform:
      img = self.transform(img)

    return img

In [ ]:

# Spatial size of training images, images are resized to this size.
image_size = 64
# Transformations to be applied to each individual image sample
transform=transforms.Compose([
   transforms.CenterCrop(140),
   transforms.Resize(image_size),
  
    transforms.ToTensor(),
 
])
# Load the dataset from file and apply transformations
celeba_dataset = CelebADataset(root, transform)

 



In [ ]:
import torchvision.utils as vutils
 

In [ ]:
celeba_dataloader = torch.utils.data.DataLoader(celeba_dataset,shuffle=False,batch_size=len(celeba_dataset))

In [ ]:
!nvidia-smi

In [ ]:
X=next(iter(celeba_dataloader))

In [ ]:
import shutil

shutil.rmtree('celeba/')
 
 

In [ ]:
X=X.numpy()
X=X.transpose([0,2,3,1])

In [ ]:
n_train=180000
n_test=X.shape[0]-n_train
x_train=X[0:n_train,:,:,:]
x_test=X[n_train:X.shape[0],:,:,:]

In [ ]:
import gc
del(X)
del(celeba_dataset)
 
gc.collect()

In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],64*64*3))

In [ ]:
from sklearn.cluster import MiniBatchKMeans
kmeans = sklearn.cluster.MiniBatchKMeans(n_clusters=center,batch_size=1000)
kmeans = kmeans.fit(x_train)
A=kmeans.cluster_centers_

In [ ]:
nn=x_train.shape[0]
gg=zeros((center,nn))
for i in range(nn):
  for j in range(center):
    gg[j,i]=np.dot(np.transpose(x_train[i,:]-A[j,:]),(x_train[i,:]-A[j,:]))
label=zeros((nn,1))
for i in range(nn):
  label[i,0]=np.argmin(gg[:,i])

radius=zeros((center,1))
for i in range(center):
  radius[i,0]=np.max(gg[i,np.where(label[:,0]==i)])**0.5+1


In [ ]:
 
if pou=='smooth':
  def rho(xindex,r,cindex,gg):
    z=gg[cindex,xindex]**0.5
    if r>=z:
      return (r**2-z**2)**gamma
    else:
      return 0

  def trho(index,xindex,gg):
    z=0
    for j in range(center):
      z=z+rho(xindex,radius[j,0],j,gg)
    return rho(xindex,radius[index,0],index,gg)/z
  prob=zeros((center,1))
  for j in range(center):
    for i in  range(nn):
      prob[j,0]=prob[j,0]+trho(j,i,gg)/nn     

if pou=='indicator':
   prob=zeros((center,1))
   for j in range(center):
        prob[j,0]=len(np.where(label==j)[0])/nn



In [ ]:
nprob=empty((center,1))
for j in range(center):
   nprob[j,0]=round(prob[j,0]*100)
nprob[np.argmax(nprob)]=nprob[np.argmax(nprob)]+100-sum(nprob)
nnprob=empty((center,1))
for j in range(center):
   nnprob[j,0]=round(prob[j,0]*10000)

In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],64,64,3))

In [ ]:
latent_dim = 64
intermediate_dim=8*8*1024
 
input_dim=64
n_channel=3
 


In [ ]:
#RBF kernel
# def compute_kernel(x, y):
#     x_size = tf.shape(x)[0]
#     y_size = tf.shape(y)[0]
#     dim = tf.shape(x)[1]
#     depth = tf.shape(x)[2]
#     tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim,depth])), tf.stack([1, y_size, 1,1]))
#     tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim,depth])), tf.stack([x_size, 1, 1,1]))
#     return tf.exp(-tf.reduce_sum(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))
  
#IMQ kernel
def compute_kernel(x, y,h):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
   
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1,]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return 2*h*tf.cast(dim, tf.float32)/(2*h*tf.cast(dim, tf.float32)+tf.reduce_sum(tf.square(tiled_x - tiled_y), axis=2))


def compute_mmd(x, y):
    stat=0
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    for scale in [1]:
        h=scale
        x_kernel = compute_kernel(x, x,h)-tf.eye(x_size)
        y_kernel = compute_kernel(y, y,h)-tf.eye(y_size)
        xy_kernel = compute_kernel(x, y,h)
        stat=stat+ tf.cast(x_size/(x_size-1),'float32')*tf.reduce_mean(x_kernel)  + tf.cast(y_size/(y_size-1),'float32')*tf.reduce_mean(y_kernel)  - 2 * tf.reduce_mean(xy_kernel)
    return stat


In [ ]:
import numpy as np

def sample_spherical(npoints, ndim):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec

In [ ]:
def compute_swass(x,y):
  val=0
  direction=sample_spherical(30,latent_dim)
  x=tf.matmul(x,direction)
  y=tf.matmul(y,direction)
  x1=tf.sort(x,axis=0)
  y1=tf.sort(y,axis=0)
  
  val=val+tf.reduce_mean(tf.abs(x1-y1))
  #val=val+keras.metrics.mean_squared_error(K.flatten(x1),K.flatten(y1))

  return val

In [ ]:

encoder_inputs = keras.Input(shape=(64, 64, 3))
x=layers.Conv2D(128, 5,strides=2,padding="same")(encoder_inputs)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(256, 5,strides=2,padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(512, 5,strides=2,padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(1024, 5,strides=2,padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
encoder_outputs = layers.Flatten()(x)
 
 
encoder_cov = keras.Model(encoder_inputs, encoder_outputs, name="encoder_cov")
encoder_cov.summary()



encoder_linear_inputs=keras.Input(shape=(16384,))
encoder_linear_outputs = layers.Dense(latent_dim*center)(encoder_linear_inputs)
encoder_linear = keras.Model(encoder_linear_inputs, encoder_linear_outputs, name="encoder_linear")
encoder_linear.summary()
 





 

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
decoder_linear_outputs = layers.Dense(intermediate_dim*center, activation="relu")(latent_inputs)
decoder_linear = keras.Model(latent_inputs, decoder_linear_outputs, name="decoder_linear")
decoder_linear.summary()
decoder_inputs= keras.Input(shape=(intermediate_dim,))
x = layers.Reshape((8, 8, 1024))(decoder_inputs)
x = layers.Conv2DTranspose(512, 5, strides=2, padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2DTranspose(256, 5, strides=2, padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2DTranspose(128, 5, strides=2, padding="same")(x)
x= layers.BatchNormalization()(x)
x = layers.ReLU()(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, strides=1,activation="tanh", padding="same")(x)
decoder_cov = keras.Model(decoder_inputs, decoder_outputs, name="decoder_cov")
decoder_cov.summary()
  

In [ ]:

  
class VAE(keras.Model):
    def __init__(self, encoder_cov,encoder_linear, decoder_cov,decoder_linear, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder_cov = encoder_cov
        self.decoder_cov = decoder_cov
        self.encoder_linear = encoder_linear
        self.decoder_linear = decoder_linear
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.mmd_loss_tracker = keras.metrics.Mean(name="mmd_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.mmd_loss_tracker,
        ]

    def train_step(self, data):
        if pou=='smooth':
          data1=tf.reshape(data,(batch_size,64*64*3))
          rhovalue=[]
          for i in range(center):
            a=tf.sqrt(tf.reduce_sum(tf.square(data1-tf.transpose(tf.repeat(tf.expand_dims(A[i,],-1),data.shape[0],axis=1))),axis=1))
            rhovalue.append(tf.math.multiply(tf.cast(tf.where(radius[i,0]>=a, 1, 0),'float32'),(radius[i,0]**2-a**2)**gamma))
          b=tf.reduce_sum(tf.stack(rhovalue),axis=0)
          for i in range(center):
            rhovalue[i]=rhovalue[i]/b
          x=[]
          batch=[]
          for centerite in range(center):
              kk=tf.where(tf.where(tf.random.uniform([data.shape[0]])<rhovalue[centerite],0,1)==0) 
              x.append(tf.gather_nd(data, kk))
              batch.append(kk.shape[0])
      
        if pou=='indicator':
          data1,label=data
          x=[]
          batch=[]
          for centerite in range(center):
              kk=tf.where(label[:,0]==centerite)
              x.append(tf.gather_nd(data1, kk))
              batch.append(kk.shape[0])
      
        x=tf.concat(x,axis=0)
        with tf.GradientTape() as tape:
            meanlist=encoder_linear(encoder_cov(x))
            meanlist=tf.split(meanlist,batch,axis=0)
            for i in range(center):
                    meanlist[i]=meanlist[i][:,(i*latent_dim):((i+1)*latent_dim)]
            meanlistconcat=tf.concat(meanlist,axis=0)
            predsini=decoder_linear(meanlistconcat)
            predsini=tf.split(predsini,batch,axis=0)
            for i in range(center):
                    predsini[i]=predsini[i][:,(i*intermediate_dim):((i+1)*intermediate_dim)]
            predsini=tf.concat(predsini,axis=0)
            predslist=decoder_cov(predsini)
            x=tf.split(x,batch,axis=0)
            predslist=tf.split(predslist,batch,axis=0)
            reconstruction_loss=0
            for i in range(center):
                      reconstruction_loss=reconstruction_loss+input_dim * input_dim * keras.metrics.mean_squared_error(K.flatten(x[i]), K.flatten(predslist[i]))*prob[i]
            mmd_loss=0
            for i in range(center):
              if is_copied==1:
                  meanlist[i]=tf.concat([meanlist[i],meanlist[i]],axis=0)
             
              add_meanlist=bandwidth*tf.random.normal(shape =[meanlist[i].shape[0], latent_dim])
              meanlist[i]=meanlist[i]+ add_meanlist
              if penalty=="MMD":
                  mmd_loss= mmd_loss+100/center*compute_mmd(tf.random.normal(shape = [meanlist[i].shape[0], latent_dim]), meanlist[i])
              if penalty=="SWASS":
                  mmd_loss= mmd_loss+30/center*compute_swass(tf.random.normal(shape = [meanlist[i].shape[0], latent_dim]), meanlist[i])
             
            
            total_loss=K.mean(reconstruction_loss + mmd_loss)
              
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.mmd_loss_tracker.update_state(mmd_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "mmd_loss": self.mmd_loss_tracker.result(),
        }


In [ ]:
x_train=x_train.reshape(x_train.shape[0],input_dim,input_dim,n_channel)
    
x_train=x_train[0:179967,:,:,:]
label=label[0:179967,:]
 

In [ ]:
from keras.callbacks import LearningRateScheduler
def decay_schedule(epoch, lr):
    if (epoch % 30 == 0) and (epoch != 0):
        lr = lr * 0.5
    if (epoch % 50 == 0) and (epoch != 0):
        lr = lr * 0.2
    return lr

lr_scheduler = LearningRateScheduler(decay_schedule)

In [ ]:
 
vae = VAE(encoder_cov, encoder_linear,decoder_cov,decoder_linear)

vae.compile(optimizer=keras.optimizers.Adam(0.001),run_eagerly=True)

if pou=='indicator':
 vae.fit(x_train,label,epochs=56, batch_size=batch_size,callbacks=[lr_scheduler])
if pou=='smooth':
 A=A.astype('float32')
 vae.fit(x_train,epochs=56, batch_size=batch_size)
    


# Evaluation 


In [ ]:
###generate image#######

z_sample=[]
for i in range(center):
  z_sample.append(np.random.normal(size=(int(nprob[i]), latent_dim)))
a=decoder_cov(decoder_linear(z_sample[0])[:,(0*intermediate_dim):((0+1)*intermediate_dim)])
for i in range(1,center):
  a=tf.concat([decoder_cov(decoder_linear(z_sample[i])[:,(i*intermediate_dim):((i+1)*intermediate_dim)]),a],axis=0)

a=tf.random.shuffle(a) 
 
a=a.numpy() 
n = 8
img_size = 64
img_chns=3
figure = np.zeros((img_size * n, img_size * n, img_chns))
o=0
for i in range(n):
    for j in range(n):
        x_decoded =a[o,:,:,:]
        image_grid = vutils.make_grid(torch.Tensor(x_decoded),
                              padding=2,
                              normalize=True)
        img = x_decoded.reshape(img_size, img_size, img_chns)
        figure[i * img_size: (i + 1) * img_size,j * img_size: (j + 1) * img_size] = img
        o=o+1
         


plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()


In [ ]:
#### Compute FID  and KID###

##the codes are from the book ``Generative adversarial networks with python: deep learning generative models for image synthesis and image translation'' by Brownlee, Jason and the official implementation of the paper ``Demystifying MMD GANs'' by Mikołaj Binkowsk et.al.

In [ ]:
 
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
 
from scipy.linalg import sqrtm
 
from keras.datasets.mnist import load_data
 
 
 
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

In [ ]:

from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from numpy import asarray
import gc
def calculate_fid_score(images,x_test1,n_split1=10, n_split2=10, eps=1E-16):
    # load inception v3 model
  model =InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
    # enumerate splits of images/predictions
 
 
  act1=[]
  act2=[]
  for i in range(n_split1):
      n_part = floor(images.shape[0] / n_split1)
      ix_start = i* n_part
      ix_end =(i+1)* n_part
      subset = images[ix_start:ix_end]
     
      subset = subset.astype('float32')
    
      
      # scale images to the required size
      subset = scale_images(subset, (299,299,3))
      subset = preprocess_input(subset)
      act1.append(model.predict(subset))
      
      del(subset)
      gc.collect()
      print(i)
  for i in range(n_split2):
      n_part = floor(x_test1.shape[0] / n_split2)
      ix_start = i* n_part
      ix_end =(i+1)* n_part 
      subset1 = x_test1[ix_start:ix_end]
      subset1 = subset1.astype('float32')
      subset1 = scale_images(subset1, (299,299,3))
      subset1 = preprocess_input(subset1)
      act2.append(model.predict(subset1))
     
      del(subset1)
      gc.collect()
      print(i)
  # # calculate sum squared difference between means
  act1=tf.concat(act1,axis=0)
  act2=tf.concat(act2,axis=0)
  act1=np.array(act1)
  act2=np.array(act2)
  mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
  ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
  if iscomplexobj(covmean):
          covmean = covmean.real
    # calculate score
  fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
  print(fid)
      
  return fid

In [ ]:


from __future__ import division, print_function

import os.path, sys, tarfile
import numpy as np
from scipy import linalg
from six.moves import range, urllib
from sklearn.metrics.pairwise import polynomial_kernel
import tensorflow as tf
from tqdm import tqdm

def polynomial_mmd(codes_g, codes_r, degree=3, gamma=None, coef0=1,
                   var_at_m=None, ret_var=True):
    # use  k(x, y) = (gamma <x, y> + coef0)^degree
    # default gamma is 1 / dim
    X = codes_g
    Y = codes_r

    K_XX = polynomial_kernel(X, degree=degree, gamma=gamma, coef0=coef0)
    K_YY = polynomial_kernel(Y, degree=degree, gamma=gamma, coef0=coef0)
    K_XY = polynomial_kernel(X, Y, degree=degree, gamma=gamma, coef0=coef0)

    return _mmd2_and_variance(K_XX, K_XY, K_YY,
                              var_at_m=var_at_m, ret_var=ret_var)


def _sqn(arr):
    flat = np.ravel(arr)
    return flat.dot(flat)


def _mmd2_and_variance(K_XX, K_XY, K_YY, unit_diagonal=False,
                       mmd_est='unbiased', block_size=1024,
                       var_at_m=None, ret_var=False):
    m = K_XX.shape[0]
    assert K_XX.shape == (m, m)
    assert K_XY.shape == (m, m)
    assert K_YY.shape == (m, m)
    if var_at_m is None:
        var_at_m = m

    # Get the various sums of kernels that we'll use
    # Kts drop the diagonal, but we don't need to compute them explicitly
    if unit_diagonal:
        diag_X = diag_Y = 1
        sum_diag_X = sum_diag_Y = m
        sum_diag2_X = sum_diag2_Y = m
    else:
        diag_X = np.diagonal(K_XX)
        diag_Y = np.diagonal(K_YY)

        sum_diag_X = diag_X.sum()
        sum_diag_Y = diag_Y.sum()

        sum_diag2_X = _sqn(diag_X)
        sum_diag2_Y = _sqn(diag_Y)

    Kt_XX_sums = K_XX.sum(axis=1) - diag_X
    Kt_YY_sums = K_YY.sum(axis=1) - diag_Y
    K_XY_sums_0 = K_XY.sum(axis=0)
    K_XY_sums_1 = K_XY.sum(axis=1)

    Kt_XX_sum = Kt_XX_sums.sum()
    Kt_YY_sum = Kt_YY_sums.sum()
    K_XY_sum = K_XY_sums_0.sum()

    if mmd_est == 'biased':
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)
                + (Kt_YY_sum + sum_diag_Y) / (m * m)
                - 2 * K_XY_sum / (m * m))
    else:
        assert mmd_est in {'unbiased', 'u-statistic'}
        mmd2 = (Kt_XX_sum + Kt_YY_sum) / (m * (m-1))
        if mmd_est == 'unbiased':
            mmd2 -= 2 * K_XY_sum / (m * m)
        else:
            mmd2 -= 2 * (K_XY_sum - np.trace(K_XY)) / (m * (m-1))

    if not ret_var:
        return mmd2

    Kt_XX_2_sum = _sqn(K_XX) - sum_diag2_X
    Kt_YY_2_sum = _sqn(K_YY) - sum_diag2_Y
    K_XY_2_sum = _sqn(K_XY)

    dot_XX_XY = Kt_XX_sums.dot(K_XY_sums_1)
    dot_YY_YX = Kt_YY_sums.dot(K_XY_sums_0)

    m1 = m - 1
    m2 = m - 2
    zeta1_est = (
        1 / (m * m1 * m2) * (
            _sqn(Kt_XX_sums) - Kt_XX_2_sum + _sqn(Kt_YY_sums) - Kt_YY_2_sum)
        - 1 / (m * m1)**2 * (Kt_XX_sum**2 + Kt_YY_sum**2)
        + 1 / (m * m * m1) * (
            _sqn(K_XY_sums_1) + _sqn(K_XY_sums_0) - 2 * K_XY_2_sum)
        - 2 / m**4 * K_XY_sum**2
        - 2 / (m * m * m1) * (dot_XX_XY + dot_YY_YX)
        + 2 / (m**3 * m1) * (Kt_XX_sum + Kt_YY_sum) * K_XY_sum
    )
    zeta2_est = (
        1 / (m * m1) * (Kt_XX_2_sum + Kt_YY_2_sum)
        - 1 / (m * m1)**2 * (Kt_XX_sum**2 + Kt_YY_sum**2)
        + 2 / (m * m) * K_XY_2_sum
        - 2 / m**4 * K_XY_sum**2
        - 4 / (m * m * m1) * (dot_XX_XY + dot_YY_YX)
        + 4 / (m**3 * m1) * (Kt_XX_sum + Kt_YY_sum) * K_XY_sum
    )
    var_est = (4 * (var_at_m - 2) / (var_at_m * (var_at_m - 1)) * zeta1_est
               + 2 / (var_at_m * (var_at_m - 1)) * zeta2_est)

    return mmd2, var_est


In [ ]:
 
def calculate_kid_score(images,x_test1,n_split1=10, n_split2=10, eps=1E-16):
    # load inception v3 model
  model =InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
    # enumerate splits of images/predictions
  #scores = list()
 
  act1=[]
  act2=[]
  for i in range(n_split1):
      n_part = floor(images.shape[0] / n_split1)
      ix_start = i* n_part
      ix_end =(i+1)* n_part
      subset = images[ix_start:ix_end]
     
      subset = subset.astype('float32')
    
     
      subset = scale_images(subset, (299,299,3))
      subset = preprocess_input(subset)
      act1.append(model.predict(subset))
      
      del(subset)
      gc.collect()
      print(i)
  for i in range(n_split2):
      n_part = floor(x_test1.shape[0] / n_split2)
      ix_start = i* n_part
      ix_end =(i+1)* n_part 
      subset1 = x_test1[ix_start:ix_end]
      subset1 = subset1.astype('float32')
      subset1 = scale_images(subset1, (299,299,3))
      subset1 = preprocess_input(subset1)
      act2.append(model.predict(subset1))
     
      del(subset1)
      gc.collect()
      print(i)
   # calculate sum squared difference between means
  act1=tf.concat(act1,axis=0)
  act2=tf.concat(act2,axis=0)
  fid =polynomial_mmd(act1,act2)
      # store
  print(fid)
      
  return fid

In [ ]:
z_sample=[]
for i in range(center):
  z_sample.append(np.random.normal(size=(int(np.round(nprob[i])), latent_dim)))

a=decoder_cov(decoder_linear(z_sample[0])[:,(0*intermediate_dim):((0+1)*intermediate_dim)])
for i in range(1,center):
  a=tf.concat([decoder_cov(decoder_linear(z_sample[i])[:,(i*intermediate_dim):((i+1)*intermediate_dim)]),a],axis=0)

for k in range(1,200):
  z_sample=[]
  for i in range(center):
      z_sample.append(np.random.normal(size=(int(np.round(nprob[i])), latent_dim)))
  for i in range(center):
      a=tf.concat([decoder_cov(decoder_linear(z_sample[i])[:,(i*intermediate_dim):((i+1)*intermediate_dim)]),a],axis=0)

image=np.reshape(np.array(a),(a.shape[0],64,64,3))*255
x_test1=np.reshape(np.array(x_test[0:20000,:,:,:]),(x_test[0:20000,:,:,:].shape[0],64,64,3))*255
calculate_kid_score(image,x_test1)
#calculate_fid_score(image,x_test1)